<a href="https://colab.research.google.com/github/Hmanguyi/AiMin/blob/main/RenmyMi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting started with Mistral AI API

In [ ]:
from fastapi import FastAPI, File, UploadFile
import base64
import uvicorn
import nest_asyncio
from mistralai import Mistral

# Apply nest_asyncio for Jupyter Notebook (if running in a notebook)
nest_asyncio.apply()

app = FastAPI()

# Your Mistral API key and model
API_KEY = "6pdlTRLqX2GhuEjgrtNJqwAhBWm8Yy6M"
MODEL = "pixtral-12b-2409"
client = Mistral(api_key=API_KEY)

def encode_image(file_bytes):
    """Convert an image in bytes to base64."""
    return base64.b64encode(file_bytes).decode('utf-8')

@app.post("/classify/")
async def classify_image(file: UploadFile = File(...)):
    """Receives an image, sends it to Mistral, and returns classification."""
    try:
        # Read file as bytes first
        file_bytes = await file.read()

        # Encode image correctly
        base64_image = encode_image(file_bytes)

        # Define messages for Mistral API
        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "is it food,is it recycleable plastic? one word"
                    },
                    {
                        "type": "image_url",
                        "image_url": f"data:image/jpeg;base64,{base64_image}"
                    }
                ]
            }
        ]

        # Get response from Mistral AI
        chat_response = client.chat.complete(
            model=MODEL,
            messages=messages
        )

        # Extract classification result
        classification = chat_response.choices[0].message.content.strip()

        return {"classification": classification}

    except Exception as e:
        return {"error": str(e)}

# To run the app using Uvicorn (locally)
# uvicorn.run(app, host="0.0.0.0", port=5000)
